<a href="https://colab.research.google.com/github/efacundoargana/scrape-acto-publico-caba/blob/main/actopublico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from requests import get
import bs4 as bs
from tqdm.notebook import tqdm

def details(job):
  return job.find("a")["href"]

details_list = []
last_page = 53

for page in tqdm(range(1,last_page+1)):
  result = get(f"https://actopublico.bue.edu.ar/?page={page}")
  soup = bs.BeautifulSoup(result.content,'lxml')

  jobs = [job for job in soup.find_all('div',{"class":"card-flex"})]
  details_list += [details(job) for job in jobs]

details_list = set(details_list)

In [ ]:
import re
import pandas as pd

pd.set_option('display.max_colwidth', None)

def job_partial_data(job):
  texto = job
  texto = re.sub(r"[ \t]+?\n+", "\n", texto)
  texto = re.sub(r"n+[ \t]+?", "\n", texto)
  texto = re.sub(r"[ \t]+", " ", texto)
  texto = re.sub(r"\n+", "\n", texto)
  regla_de_busqueda = ".+:\s?\n.+"

  data = {}
  for value in re.findall(regla_de_busqueda,texto):
    try:
      a, b = value.split(":")[:2]
      data[a.strip()] = b.strip()
    except:
      pass
  return data

def job_data(job):
  data = []

  info_establecimientos = job.find_all("div",{"class":"info-establecimiento"})
  if info_establecimientos == []:
    for table in job.find_all("div",{"class":"card-flex"}):
      data.append(job_partial_data(table.text))
  else:
    for table in info_establecimientos:
      job_data = job_partial_data(table.text)
      for table in job.find_all("div",{"class":"card-flex"}):
        job_data.update(job_partial_data(table.text))
      data.append(job_data)

  return data

jobs = []
for job_link in tqdm(details_list):
  if job_link.startswith("https://actopublico.bue.edu.ar/solicitud/"):
    result = get(job_link)
    job_soup = bs.BeautifulSoup(result.content,'lxml')
    jobs += job_data(job_soup)

observations = pd.DataFrame(jobs)
observations.to_csv("acto_publico.csv",index=False)
observations.head()

,Establecimiento del Cargo,del Establecimiento del Cargo,Distrito,Caracter,Turno,Horas cátedra,Observación,Año,Horario,Docente,Causa de la solicitud,Mail de la escuela,Estado,Tipo de acto,Área,Nombre del Cargo,Asignatura,Especialidad,Fecha de Acto Público,Horas Cátedra
0,E.N.S. 4 - ESTANISLAO S. ZEBALLOS,AV. RIVADAVIA 4950 - Caballito,Distrito 8,Suplente,Varios,7,2°5° + 3 hs cod 162 2 año 2021 ( se crea,2021,Martes,"RUBINO, CLAUDIO",Lic Art 71,ens4@bue.edu.ar,Publicada,Interinatos y Suplencias,NORMALES,PROFESOR DE NIVEL MEDIO,HISTORIA,NES - CICLO BASICO DE DISEÑO CURRICULAR,jueves 13 de mayo de 2021 a las 17,7 Hrs.
1,E.N.S. 4 - ESTANISLAO S. ZEBALLOS,AV. RIVADAVIA 4950 - Caballito,Distrito 8,Suplente,Varios,7,2°5° + 3 hs cod 162 2 año 2021 ( se crea,2021,Martes,"RUBINO, CLAUDIO",Lic Art 71,ens4@bue.edu.ar,Publicada,Interinatos y Suplencias,NORMALES,PROFESOR DE NIVEL MEDIO,HISTORIA,NES - CICLO BASICO DE DISEÑO CURRICULAR,jueves 13 de mayo de 2021 a las 17,7 Hrs.
2,Escuela sup.de ens. art. e,OLIDEN 1245 - Liniers,Distrito 20,Interino,Tarde,36,.,2021,Lunes 12,BRASCO,RENUNCIA,dea_escuela_de_teatro@bue.edu.ar,Publicada,Acto público de Ascenso,ARTISTICA,SUB JEFE DE PRECEPTORES,NaN,NaN,jueves 13 de mayo de 2021 a las 16,NaN
3,ESCUELA SUP.DE ENS. ART. EN TEATRO NINI MARSHALL,OLIDEN 1245 - Liniers,Distrito 20,Interino,Tarde,NaN,.,2021,Lunes 12,NaN,NaN,NaN,Publicada,Acto público de Ascenso,ARTISTICA,SUB JEFE DE PRECEPTORES,NaN,NaN,NaN,36 Hrs.
4,E.n.s. 3 - bernardino rivadavia,BOLÍVAR 1235-NIVEL INICIAL PERÚ 1236 - Sa,Distrito 4,Suplente,Mañana,30,Docente,2021,8,NaN,COVID-19,ens3@bue.edu.ar,Publicada,Interinatos y Suplencias,NORMALES,MAESTRO AUXILIAR J.I. (E.S.),NaN,NaN,jueves 13 de mayo de 2021 a las 18,NaN


In [ ]:
observations.shape

(338, 20)